In [1]:
#!pip install -U scikit-learn


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder

from scipy.special import softmax
#import pandas.core.common import flatten

In [6]:
no_of_robot_steps = 100

fixed_wheel_x = np.loadtxt(str(no_of_robot_steps) + "_steps_fixed_xs.csv", delimiter=',')
fixed_wheel_y = np.loadtxt(str(no_of_robot_steps) + "_steps_fixed_ys.csv", delimiter=',')
fixed_wheel_theta = np.loadtxt(str(no_of_robot_steps) + "_steps_fixed_thetas.csv", delimiter=',')

omni_wheel_x = np.loadtxt(str(no_of_robot_steps) + "_steps_omni_xs.csv", delimiter=',')
omni_wheel_y = np.loadtxt(str(no_of_robot_steps) + "_steps_omni_ys.csv", delimiter=',')
omni_wheel_theta = np.loadtxt(str(no_of_robot_steps) + "_steps_omni_thetas.csv", delimiter=',')

two_wheel_x = np.loadtxt(str(no_of_robot_steps) + "_steps_two_fixed_xs.csv", delimiter=',')
two_wheel_y = np.loadtxt(str(no_of_robot_steps) + "_steps_two_fixed_ys.csv", delimiter=',')
two_wheel_theta = np.loadtxt(str(no_of_robot_steps) + "_steps_two_fixed_thetas.csv", delimiter=',')


len_of_steps = len(fixed_wheel_x[0])
total_len_of_columns = len_of_steps * 3
total_pose_size = len(fixed_wheel_x.flatten()) + len(fixed_wheel_y.flatten()) + len(fixed_wheel_theta.flatten())
total_pose_size = int(total_pose_size/total_len_of_columns)

#print("Number of robot steps: ", len_of_steps, "\nLenght of columns for steps (x,y,theta): ",total_len_of_columns, "\nTotal number of samples: ", total_pose_size)

fixed_wheel_poses = np.column_stack((fixed_wheel_x.flatten(),fixed_wheel_y.flatten(),fixed_wheel_theta.flatten()))
fixed_wheel_poses = np.reshape(fixed_wheel_poses,(total_pose_size,total_len_of_columns))


omni_wheel_poses = np.column_stack((omni_wheel_x.flatten(),omni_wheel_y.flatten(),omni_wheel_theta.flatten()))
omni_wheel_poses = np.reshape(omni_wheel_poses,(total_pose_size,total_len_of_columns))

two_fixed_wheel_poses = np.column_stack((two_wheel_x.flatten(),two_wheel_y.flatten(),two_wheel_theta.flatten()))
two_fixed_wheel_poses = np.reshape(two_fixed_wheel_poses,(total_pose_size,total_len_of_columns))

x_headings = ["step x " + str(i+1) for i in range(int(total_len_of_columns/3))]
y_headings = ["step y " + str(i+1) for i in range(int(total_len_of_columns/3))]
theta_headings = ["step theta " + str(i+1) for i in range(int(total_len_of_columns/3))]

headings = []
for i in range(0,len(x_headings)):
    headings.append(x_headings[i])
    headings.append(y_headings[i])
    headings.append(theta_headings[i])
#headings = x_headings + y_headings + theta_headings
#headings

fixed_p = pd.DataFrame(fixed_wheel_poses, columns = headings)
omni_p = pd.DataFrame(omni_wheel_poses, columns = headings)
two_fixed_p = pd.DataFrame(two_fixed_wheel_poses, columns = headings)

#Add class labels
fixed_p.insert(0,'Model type', 'fw')
omni_p.insert(0,'Model type', 'ow')
two_fixed_p.insert(0,'Model type', 'tf')

wheel_poses = pd.concat([fixed_p, omni_p, two_fixed_p])

wheel_poses = wheel_poses.iloc[np.random.RandomState(seed=16).permutation(len(wheel_poses))]
wheel_poses = wheel_poses.reset_index(drop=True)

X = wheel_poses.drop(columns='Model type')
y = wheel_poses['Model type']

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=16)
#trprint(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

X_train, X_rem, y_train, y_rem = train_test_split(X, y, test_size = 0.3, random_state=16)

X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size = 0.5, random_state=16)




reg = LogisticRegression(random_state = 16, multi_class='multinomial')
reg.fit(X_train, y_train)
y_pred = reg.predict(X_valid)
#scorings = ['accuracy', 'neg_log_loss', 'roc_auc', 'recall', 'precision', 'f1']
scorings = ['accuracy', 'neg_log_loss']


the_mean = []
the_std = []
the_param = []
for scoring in scorings:
    solvers = ['newton-cg', 'lbfgs']
    penalty = ['l2']
    c_values = [100, 10, 1.0, 0.1, 0.01]
    # define grid search
    grid = dict(solver=solvers,penalty=penalty,C=c_values)
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    grid_search = GridSearchCV(estimator=reg, param_grid=grid, n_jobs=-1, cv=cv, scoring=scoring,error_score=0, verbose = 4, return_train_score=True)
    grid_result = grid_search.fit(X_train, y_train)
    # summarize results
    print("Results for ", scoring)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r\n\n" % (mean, stdev, param))
    means_test = grid_result.cv_results_['mean_test_score']
    means_train = grid_result.cv_results_['mean_train_score']
    
    
    
    plt.figure(figsize=(10,15))
    fig, ax1 = plt.subplots()
    
    
    color1 = 'tab:green'
    color2 = 'tab:red'
    ax1.set_ylabel('Test', color = color1)
    ax1.plot(means_test, color = color1, label = 'test')
    ax1.plot(means_train, color = color2, label = 'train')
    ax1.tick_params(axis ='y', labelcolor = color1)
    
    fig.suptitle('Mean Test vs Mean Train Logistic Regression - ' + str(scoring), fontweight ="bold")
    plt.show()

    
    
    
    

C:\Users\ayoid\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


OSError: [Errno 22] Invalid argument

In [ ]:
#Using the tuned parameter values for the best log loss
#Best: -1.106958 using {'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}
reg_best_log = LogisticRegression(multi_class='multinomial',C=100, penalty='l2', solver='lbfgs')
reg_best_log.fit(X_train, y_train)
y_pred_best_log = reg_best_log.predict(X_test)
confusion_matrix_best_log = metrics.plot_confusion_matrix(reg_best_log, X_test, y_test)
print("Accuracy: ", metrics.accuracy_score(y_test, y_pred_best_log))
print("Log Loss: ", metrics.zero_one_loss(y_test, y_pred_best_log))
print(confusion_matrix_best_log)

#The log loss is high and accuracy is low


In [ ]:
#Using the tuned parameter values for the best accuracy
#Best: -1.106958 using {'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}
reg_best_log = LogisticRegression(random_state = 16, multi_class='multinomial', C=0.01, penalty='l2', solver='newton-cg')
reg_best_log.fit(X_train, y_train)
y_pred_best_log = reg_best_log.predict(X_test)
confusion_matrix_best_log = metrics.plot_confusion_matrix(reg_best_log, X_test, y_test)
print("Accuracy: ", metrics.accuracy_score(y_test, y_pred_best_log))
print("Log Loss: ", metrics.zero_one_loss(y_test, y_pred_best_log))
print(confusion_matrix_best_log)

#We have a considerable log loss with accuracy balance.


In [ ]:
#And we run the number of steps with our best model to see how the number of steps 
#improbves the performance of the model
def get_step_results(no_of_robot_steps):
    fixed_wheel_x = np.loadtxt(str(no_of_robot_steps) + "_steps_fixed_xs.csv", delimiter=',')
    fixed_wheel_y = np.loadtxt(str(no_of_robot_steps) + "_steps_fixed_ys.csv", delimiter=',')
    fixed_wheel_theta = np.loadtxt(str(no_of_robot_steps) + "_steps_fixed_thetas.csv", delimiter=',')

    omni_wheel_x = np.loadtxt(str(no_of_robot_steps) + "_steps_omni_xs.csv", delimiter=',')
    omni_wheel_y = np.loadtxt(str(no_of_robot_steps) + "_steps_omni_ys.csv", delimiter=',')
    omni_wheel_theta = np.loadtxt(str(no_of_robot_steps) + "_steps_omni_thetas.csv", delimiter=',')

    two_wheel_x = np.loadtxt(str(no_of_robot_steps) + "_steps_two_fixed_xs.csv", delimiter=',')
    two_wheel_y = np.loadtxt(str(no_of_robot_steps) + "_steps_two_fixed_ys.csv", delimiter=',')
    two_wheel_theta = np.loadtxt(str(no_of_robot_steps) + "_steps_two_fixed_thetas.csv", delimiter=',')

    #Get parameters of the dataframe we will need to build
    len_of_steps = len(fixed_wheel_x[0])
    total_len_of_columns = len_of_steps * 3
    total_pose_size = len(fixed_wheel_x.flatten()) + len(fixed_wheel_y.flatten()) + len(fixed_wheel_theta.flatten())
    total_pose_size = int(total_pose_size/total_len_of_columns)

    #print("Number of robot steps: ", len_of_steps, "\nLenght of columns for steps (x,y,theta): ",total_len_of_columns, "\nTotal number of samples: ", total_pose_size)

    #Expose and reshape the measurements in the respective wheels 
    #to build a dataframe for the wheeltype
    fixed_wheel_poses = np.column_stack((fixed_wheel_x.flatten(),fixed_wheel_y.flatten(),fixed_wheel_theta.flatten()))
    fixed_wheel_poses = np.reshape(fixed_wheel_poses,(total_pose_size,total_len_of_columns))

    omni_wheel_poses = np.column_stack((omni_wheel_x.flatten(),omni_wheel_y.flatten(),omni_wheel_theta.flatten()))
    omni_wheel_poses = np.reshape(omni_wheel_poses,(total_pose_size,total_len_of_columns))
    
    two_fixed_wheel_poses = np.column_stack((two_wheel_x.flatten(),two_wheel_y.flatten(),two_wheel_theta.flatten()))
    two_fixed_wheel_poses = np.reshape(two_fixed_wheel_poses,(total_pose_size,total_len_of_columns))

    #Creating the column names. Each measurement contains a X,y and theta value
    x_headings = ["step x " + str(i+1) for i in range(int(total_len_of_columns/3))]
    y_headings = ["step y " + str(i+1) for i in range(int(total_len_of_columns/3))]
    theta_headings = ["step theta " + str(i+1) for i in range(int(total_len_of_columns/3))]
    headings = []
    for i in range(0,len(x_headings)):
        headings.append(x_headings[i])
        headings.append(y_headings[i])
        headings.append(theta_headings[i])
    #headings = x_headings + y_headings + theta_headings
    #headings

    #Creating the respective wheel dataframes
    fixed_p = pd.DataFrame(fixed_wheel_poses, columns = headings)
    omni_p = pd.DataFrame(omni_wheel_poses, columns = headings)
    two_fixed_p = pd.DataFrame(two_fixed_wheel_poses, columns = headings)

    #Tagging class labels in a new column with the wheel type 
    #and joining the dataframes into one, we shuffle the dataframe
    #and reset the index
    fixed_p.insert(0,'Model type', 'fw')
    omni_p.insert(0,'Model type', 'ow')
    two_fixed_p.insert(0,'Model type', 'tf')
    wheel_poses = pd.concat([fixed_p, omni_p, two_fixed_p])
    wheel_poses = wheel_poses.iloc[np.random.RandomState(seed=16).permutation(len(wheel_poses))]
    wheel_poses = wheel_poses.reset_index(drop=True)

    X = wheel_poses.drop(columns='Model type')
    y = wheel_poses['Model type']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=16)
    #trprint(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    
    #X_train, X_rem, y_train, y_rem = train_test_split(X, y, test_size = 0.3, random_state=16)  
    #X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size = 0.5, random_state=16)
    
    
    #Best: -1.106958 using {'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}
    
    #Using our best logger hyper parameters
    
    reg = LogisticRegression(multi_class='multinomial', C=0.01, penalty='l2', solver='newton-cg')
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)

    dic_acc = {}
    dic_zol = {}
    
    #confusion_matrix = metrics.plot_confusion_matrix(reg, X_test, y_test)
    #print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
    #print(confusion_matrix)

    y_pred_prob = reg.predict_proba(X_test)[:,1]
    
    #print(metrics.classification_report(y_test, y_pred))

    dic_acc['ACC'] = round(metrics.accuracy_score(y_test, y_pred)*100,2)
    dic_zol['Zero One Loss'] = round(metrics.zero_one_loss(y_test, y_pred),2)

    model_name = str(len_of_steps) + '_steps_LR_robot_model_weights.sav' 
    pickle.dump(reg, open(model_name, 'wb'))
    
    return dic_acc, dic_zol


steps = [5, 10, 20, 50, 100]
#steps = [100, 500, 1000, 2000, 5000, 10000]

step_dic = {}
#Collects the accuracy and log loss for each step
#steps = [5]

for no_of_robot_steps in steps:
    step_acc, step_zol = get_step_results(no_of_robot_steps)
    step_dic[no_of_robot_steps] = [step_acc, step_zol]
    
k_list = []
v_list = []
z_list = []
for k, v in step_dic.items():
    k_list.append(k) 
    v_list.append(list(v[0].values())[0])
    z_list.append(list(v[1].values())[0])
    #print("For step: ", k, "the results are: ", v )

plt.figure(figsize=(10,15))
fig, ax1 = plt.subplots()

color = 'tab:green'
ax1.set_ylabel('Accuracy', color = color)
ax1.plot(k_list, v_list, color = color)
ax1.tick_params(axis ='y', labelcolor = color)
fig.suptitle('Number of steps vs Logistics Regreesion Model accuracy\n', fontweight ="bold")
plt.show()

fig2, ax1 = plt.subplots()

color = 'tab:green'
ax1.set_ylabel('Log Loss', color = color)
ax1.plot(k_list, z_list, color = color)
ax1.tick_params(axis ='y', labelcolor = color)
fig2.suptitle('Number of steps vs Logistics Regression Model Log Loss\n', fontweight ="bold")
plt.show()




    

    


In [ ]:
#We can propose that the model's performance improves with more steps taken by the robot. 
step_dic